In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from colorcloud.behley2019iccv import SemanticKITTIDataset, UnfoldingProjection, ProjectionTransform, ProjectionToTensorTransform
from colorcloud.biasutti2019riu import RIUNet
from colorcloud.chen2020mvlidarnet import MVLidarNet
from torchvision.transforms import v2
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss, BatchNorm2d
from torch.optim import Adam
import lightning as L

In [3]:
class LitModel(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.bn = BatchNorm2d(5, affine=False, momentum=None)
        self.net = MVLidarNet()
        self.loss_fn = CrossEntropyLoss(reduction='none')

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        img, label, mask = batch
        label[~mask] = 0

        # Compute prediction error
        pred = self.net(img)
        loss = self.loss_fn(pred, label)
        loss = loss[mask].mean()
        self.log("loss", loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = Adam(self.parameters(), lr=1e-3)
        return optimizer

In [4]:
data_path = '../../data'
ds = SemanticKITTIDataset(data_path)

remapping_rules = {
    1: 1,
    4: 2,
    6: 3,
    7: 4,
    2: 4,
    9: 5,
    11: 6
}
ds.learning_remap(remapping_rules)

proj = UnfoldingProjection(W=1024, H=64)
tfms = v2.Compose([
    ProjectionTransform(proj),
    ProjectionToTensorTransform(),
])
ds.set_transform(tfms)

bs = 10
dl = DataLoader(ds, bs, num_workers=8)

In [5]:
# train model
trainer = L.Trainer(max_epochs=1)
trainer.fit(model=LitModel(), train_dataloaders=dl)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
You are using a CUDA device ('NVIDIA GeForce RTX 3080') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.

Training: |                                                                                                   …

`Trainer.fit` stopped: `max_epochs=1` reached.
